In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("Advertising.csv")

In [4]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [5]:
X = df.drop("sales", axis=1)

In [6]:
y = df["sales"]

In [7]:
# Converting to 3rd degree polynomial features
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [8]:
poly_converter = PolynomialFeatures(degree=3, include_bias=False)

In [9]:
poly_features = poly_converter.fit_transform(X)

In [10]:
X.shape

(200, 3)

In [11]:
poly_features.shape

(200, 19)

#### Splitting the *poly_features* into train and test data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [14]:
X_train.shape

(140, 19)

In [15]:
X_test.shape

(60, 19)

In [16]:
X_train[0]

array([1.93200000e+02, 1.84000000e+01, 6.57000000e+01, 3.73262400e+04,
       3.55488000e+03, 1.26932400e+04, 3.38560000e+02, 1.20888000e+03,
       4.31649000e+03, 7.21142957e+06, 6.86802816e+05, 2.45233397e+06,
       6.54097920e+04, 2.33555616e+05, 8.33945868e+05, 6.22950400e+03,
       2.22433920e+04, 7.94234160e+04, 2.83593393e+05])

In [17]:
X_test[0]

array([7.47000000e+01, 4.94000000e+01, 4.57000000e+01, 5.58009000e+03,
       3.69018000e+03, 3.41379000e+03, 2.44036000e+03, 2.25758000e+03,
       2.08849000e+03, 4.16832723e+05, 2.75656446e+05, 2.55010113e+05,
       1.82294892e+05, 1.68641226e+05, 1.56010203e+05, 1.20553784e+05,
       1.11524452e+05, 1.03171406e+05, 9.54439930e+04])

#### Applying feature-scaling independently on the train and test datasets so as to avoid data leakage

In [25]:
scaler = StandardScaler()

In [26]:
scaler.fit(X_train)

StandardScaler()

In [27]:
X_train = scaler.transform(X_train)

In [28]:
scaler.fit(X_test)

StandardScaler()

In [29]:
X_test = scaler.transform(X_test)

In [30]:
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In the above, we can see all the values are standardized

In [31]:
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

### Ridge regression
L2 regularization
- Introduces an error term proportional to the square of the coefficients
- The error is also called shrinkage penalty

In [32]:
from sklearn.linear_model import Ridge

In [33]:
ridge_model = Ridge(alpha=10)

In [34]:
type(ridge_model)

sklearn.linear_model._ridge.Ridge

In [35]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [36]:
ridge_model_predictions = ridge_model.predict(X_test)

In [37]:
type(ridge_model_predictions)

numpy.ndarray

In [38]:
ridge_model_predictions.shape

(60,)

In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [40]:
MAE = mean_absolute_error(y_test, ridge_model_predictions)
MAE

0.9670410375853897

In [41]:
RMSE = np.sqrt(mean_squared_error(y_test, ridge_model_predictions))
RMSE

1.332761154538627

#### Using cross validation along with ridge regression (L2 regularization)
This is to get the best alpha value (or lambda value)

In [42]:
from sklearn.linear_model import RidgeCV
#By default, it performs efficient Leave-One-Out Cross-Validation

In [43]:
#help(RidgeCV())

In [44]:
from sklearn.metrics import SCORERS

In [45]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [46]:
# cv = None means leave-one-out cross-validation
# This can be expensive computation for large datasets, for this case, its fine since we have only 140 rows in the training dataset (which includes the validation dataset)
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error', cv=None)

In [47]:
# The below code performs cross validation in the X_train dataset
# There is automatic separation of train and validation data within the X_train dataset
# X_test is the hold out test dataset for us
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [48]:
ridge_cv_model_predictions = ridge_cv_model.predict(X_test)

In [49]:
ridge_cv_model.alphas

array([ 0.1,  1. , 10. ])

In [50]:
# Best performing alpha
ridge_cv_model.alpha_

0.1

In [51]:
# Performance metrics on the hold-out test dataset for reporting purpose
MAE = mean_absolute_error(y_test, ridge_cv_model_predictions)
MAE

0.9873965312401293

In [52]:
RMSE = np.sqrt(mean_squared_error(y_test, ridge_cv_model_predictions))
RMSE

1.1676689886078317

In [53]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

As we can observe, ridge cross validation regression model has performed better than our ridge regression